# Model Training

## Requirements

In [3]:
## Load requirements
import pandas as pd
import plotly
import gc
import pickle
import matplotlib.pyplot as plt 

from typing import Tuple
from datetime import timedelta, datetime

from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import pmdarima as pm

from utils import load

IndentationError: expected an indented block after function definition on line 29 (utils.py, line 30)

## Data Load

In [82]:
csv_path = "../stock_forecasting/stock_data/Compiled/all_stocks_2006-01-01_to_2018-01-01.csv"
df = pd.read_csv(csv_path)
df

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,77.76,79.35,77.24,79.11,3117200,MMM
1,2006-01-04,79.49,79.49,78.25,78.71,2558000,MMM
2,2006-01-05,78.41,78.65,77.56,77.99,2529500,MMM
3,2006-01-06,78.64,78.90,77.64,78.63,2479500,MMM
4,2006-01-09,78.50,79.83,78.46,79.02,1845600,MMM
...,...,...,...,...,...,...,...
93607,2017-12-22,71.42,71.87,71.22,71.58,10979165,AABA
93608,2017-12-26,70.94,71.39,69.63,69.86,8542802,AABA
93609,2017-12-27,69.77,70.49,69.69,70.06,6345124,AABA
93610,2017-12-28,70.12,70.32,69.51,69.82,7556877,AABA


In [83]:
df['Date']=pd.to_datetime(df['Date'])
resample_df = df.set_index("Date")
print ("\nMissing values :  ", df.isnull().any())


Missing values :   Date      False
Open       True
High       True
Low        True
Close     False
Volume    False
Name      False
dtype: bool


In [84]:
df['Open']=df['Open'].fillna(method='ffill')
df['High']=df['High'].fillna(method='ffill')
df['Low']=df['Low'].fillna(method='ffill')

print ("\nMissing values :  ", df.isnull().any())


Missing values :   Date      False
Open      False
High      False
Low       False
Close     False
Volume    False
Name      False
dtype: bool


In [85]:
#resample_df = resample_df.resample("D").mean()
resample_df

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,77.76,79.35,77.24,79.11,3117200,MMM
2006-01-04,79.49,79.49,78.25,78.71,2558000,MMM
2006-01-05,78.41,78.65,77.56,77.99,2529500,MMM
2006-01-06,78.64,78.90,77.64,78.63,2479500,MMM
2006-01-09,78.50,79.83,78.46,79.02,1845600,MMM
...,...,...,...,...,...,...
2017-12-22,71.42,71.87,71.22,71.58,10979165,AABA
2017-12-26,70.94,71.39,69.63,69.86,8542802,AABA
2017-12-27,69.77,70.49,69.69,70.06,6345124,AABA


## Model Training

### ARIMA (Auto ARIMA)

In [ ]:
## iterations over stock name in DF
for stock in df["Name"].unique():
    #initialize model
    try: 
        model = pm.auto_arima(resample_df.reset_index().drop(columns=["Name"])['Close'], 
                            m=12, seasonal=True,
                        start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                            suppress_warnings=True,
                        stepwise=True, trace=True)
    except:
        print("Error in model initialization: ", stock)
        pass

    #Splitting
    try:
        df_train, df_val, df_test = slicer(resample_df[(resample_df['Name']==stock)])
    except:
        print("Error in splitting: ",stock)
        pass

    #model training & fitting
    try: 
        model.fit(df_train['Close'])
    except:
        print("Error in model fitting: ",stock)
        pass
    
    #append score... PENDING
    
    #export model to .pkl (pickle.dump)
    try: 
        file_name = (f'../stock_forecasting/AutoARIMA_Models/PKL/{stock}_autoARIMA_')
        # save
        pickle.dump(model, open(file_name + 'Model.pkl', "wb"))
    except:
        print("Error in model export: ",stock)
        pass

    gc.collect() #RAM usage optimization



In [2]:
df1, resample_df1 = load()  

NameError: name 'load' is not defined